# Figure 6: Stable isotope fractionation of volatiles during degassing

In [1]:
# import python packages
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import math

## Function to calculate isotopic composition of melt and vapor during open- and closed system degassing

In [2]:
def simple_isotope_fractionation(D,db): 
    # D = cap-delta fractionation factor between vapor and melt per mil
    # db = little-delta isotope value of bulk system in per mil
    for n in range(0,1000,1):
        F = 1. - (n/1000.) # fraction of volatile remain in the melt
        dm_closed = db - D*(1.-F) # little-delta isotope value of melt during closed-system degassing in per mil
        dv_closed = db + D*F # little-delta isotope value of vapor during closed-system degassing in per mil
        dm_open = db + D*math.log(F) # little-delta isotope value of melt during open-system degassing in per mil
        dv_open_inst = dm_open + D # little-delta isotope value of instantaneus vapor during open-system degassing in per mil
        if n == 0.:
            dv_open = dv_open_inst # little-delta isotope value of total vapor during open-system degassing in per mil
        else:
            dv_open = dv_open_inst*(1./n) + ((n-1.)/n)*dv_open # little-delta isotope value of total vapor during open-system degassing in per mil
        results1 = pd.DataFrame([[F,dm_closed,dv_closed,dm_open,dv_open_inst,dv_open]])
        if n == 0.:
            results_headers = pd.DataFrame([["F","dm_closed","dv_closed","dm_open","dv_open_inst","dv_open"]])
            results = pd.concat([results_headers, results1])
        else:
            results = pd.concat([results, results1])
    results.columns = results.iloc[0]
    results = results[1:]  
    return results

## Stable isotope data on natural samples

### Hydrogen isotope data on rhyolitic matrix glasses from Mono Craters (CA, USA) and model fit to data from Figure 6 from Newman et al. (1988)
Newman, S., Epstein, S. and Stolper, E., 1988. Water, carbon dioxide, and hydrogen isotopes in glasses from the ca. 1340 AD eruption of the Mono Craters, California: Constraints on degassing phenomena and initial volatile content. Journal of Volcanology and Geothermal Research, 35(1-2), pp.75-96. https://doi.org/10.1016/0377-0273(88)90007-8

In [3]:
newman88 = pd.read_csv("files/data_Newman+88_JVGR.csv") 

### Carbon isotope data on basaltic matrix glasses and crushed bubbles from Easter Microplate East Ridge (East Pacific Rise) from Aubaud et al. (2004)
Aubaud, C., Pineau, F., Jambon, A. and Javoy, M., 2004. Kinetic disequilibrium of C, He, Ar and carbon isotopes during degassing of mid-ocean ridge basalts. Earth and Planetary Science Letters, 222(2), pp.391-406. https://doi.org/10.1016/j.epsl.2004.03.001

In [4]:
aubaud04 = pd.read_csv("files/data_Aubaud+04_EPSL.csv")

### Sulfur isotope data on basaltic melt inclusions and matrix glasses from Kīlauea (Hawaiʻi, USA) from Lerner et al. (2021)
Lerner, A.H., Wallace, P.J., Shea, T., Mourey, A.J., Kelly, P.J., Nadeau, P.A., Elias, T., Kern, C., Clor, L.E., Gansecki, C. and Lee, R.L., 2021. The petrologic and degassing behavior of sulfur and other magmatic volatiles from the 2018 eruption of Kīlauea, Hawaiʻi: melt concentrations, magma storage depths, and magma recycling. Bulletin of Volcanology, 83(6), p.43. https://doi.org/10.1007/s00445-021-01459-y

In [5]:
lerner21 = pd.read_csv("files/data_Lerner+21_BV.csv")

## Calculate fractionation curves for different volatiles

### Cap-delta fractionation factors for different volatile species

Blank J. G. (1993) An experimental investigation of the behavior of carbon dioxide in rhyolitic melt. California Institute of Technology. https://doi.org/10.7907/tq3x-2059

Dobson, P.F., Epstein, S., and Stolper, E.M., 1989, Hydrogen isotope fractionation between coexisting vapor and silicate glasses and melts at low pressure: Geochimica et Cosmochimica Acta , v. 53p. 2723-2730. https://doi.org/10.1016/0016-7037(89)90143-9

Fiege, A., Holtz, F., Behrens, H., Mandeville, C.W., Shimizu, N., Crede, L.S. and Göttlicher, J., 2015. Experimental investigation of the S and S-isotope distribution between H2O–S±Cl fluids and basaltic melts during decompression. Chemical Geology, 393, pp.36-54. https://doi.org/10.1016/j.chemgeo.2014.11.012 

Lee, H., Moussallam, Y., Aubaud, C., Iacono–Marziano, G., Hammond, K. and Ebel, D., 2024. Carbon isotope fractionation between CO2 and carbon in silicate melts at high temperature. Geochimica et Cosmochimica Acta. https://doi.org/10.1016/j.gca.2024.07.015 

Taylor, B.E., and Westrich, H.R., 1985, Hydrogen isotope exchange and water solubility in experiments using natural rhyolite obsidian: Eos (Transactions, American Geophysical Union), supplement , v. 66p. 387.

In [6]:
D = {}
D['H2O/OH-'] = 40.66 # Dobson et al. (1989)
D['H2O/H2Omol'] = -10.45 # Taylor and Westrich (1985)
D['CO2/CO32-'] = 2.9 # Lee et al. (2024)
D['CO2/CO2mol'] = 0. # Blank (1993)
D['H2S/S2-'] = 2.50 # Feige et al. (2015)
D['SO2/S6+'] = -1.90 # Feige et al. (2015)

In [7]:
hydrogen_OH = simple_isotope_fractionation(D['H2O/OH-'],0)
hydrogen_H2Omol = simple_isotope_fractionation(D['H2O/H2Omol'],0)
carbonate = simple_isotope_fractionation(D['CO2/CO32-'],0)
sulfur_reduced = simple_isotope_fractionation(D['H2S/S2-'],0)
sulfur_oxidised = simple_isotope_fractionation(D['SO2/S6+'],0)

## Plotting

In [8]:
# Function to convert coordinates for inset of fractionation factors
def convert_inset(D):
    x1r,x2r,x1p,x2p = -20,+50,0.25,0.95
    newD = (((D-x1r)/(x2r-x1r))*(x2p-x1p))+x1p
    return newD

In [20]:
fig = make_subplots(rows=2, cols=2, shared_yaxes = False, shared_xaxes = False,vertical_spacing=0.08, horizontal_spacing=0.05)

# define curve and symbol attributes
lw = 2
syms = 7
lc1 = '#B22221' # red
lc2 = '#0160C6' # blue
lc3 = '#FF8C03' # orange
lc4 = "lightgrey"
lc5 = "darkgrey"

r = 1
c = 1
# model open and closed system melt and vapor curves for isotope fractionation
# rescale carbonate fractionation curve to show generic degassing trend
# closed-system melt
fig.add_trace(go.Scatter(mode = "lines", x=carbonate['F'], y=carbonate['dm_closed']/D['CO2/CO32-'], 
                         line_color = lc1, line_width = lw,line_dash = "solid"), row = r, col = c)
#closed-system vapor
fig.add_trace(go.Scatter(mode = "lines", x=carbonate['F'], y=carbonate['dv_closed']/D['CO2/CO32-'], 
                         line_color = lc2, line_width = lw,line_dash = "solid"), row = r, col = c)
# open-system melt
fig.add_trace(go.Scatter(mode = "lines", x=carbonate['F'], y=carbonate['dm_open']/D['CO2/CO32-'], 
                         line_color = lc1, line_width = lw,line_dash = "dash"), row = r, col = c)
# open-system instantaneous vapor
fig.add_trace(go.Scatter(mode = "lines", x=carbonate['F'], y=carbonate['dv_open_inst']/D['CO2/CO32-'], 
                         line_color = lc2, line_width = lw,line_dash = "dot"), row = r, col = c)
# open-system accumulated vapor
fig.add_trace(go.Scatter(mode = "lines", x=carbonate['F'], y=carbonate['dv_open']/D['CO2/CO32-'], 
                         line_color = lc2, line_width = lw,line_dash = "dash"), row = r, col = c)
fig.update_xaxes(title = "fraction remaining in the melt", tickformat = ".1f", dtick= 0.2, range = [0,1], row = r, col = c)
fig.update_yaxes(title = "\u03B4/\u0394", title_font=dict(family="Times new roman"), range=[-6,2], row = r, col = c)
# inset of fractionation factors
# horizontal line at 0 per mil
fig.add_trace(go.Scatter(mode = "lines", x=[0.45,0.45], y=[-5.5,-2.5], line_color = 'darkgrey', 
                         line_width = 1,line_dash = "solid"), row = r, col = c)
# draw inset box
fig.add_trace(go.Scatter(mode = "lines", x=[0.25,0.95,0.95,0.25,0.25], y=[-5.5,-5.5,-2.5,-2.5,-5.5], 
                         line_color = 'black', line_width = 1,line_dash = "solid"), row = r, col = c)
# Hydrogen
fig.add_trace(go.Scatter(mode = "lines", x=[convert_inset(D['H2O/H2Omol']),convert_inset(D['H2O/OH-'])], y=[-3.,-3.], 
                         line_color = lc2, line_width = lw,line_dash = "solid"), row = r, col = c)
fig.add_trace(go.Scatter(mode = "markers", x=[convert_inset(D['H2O/H2Omol']),convert_inset(D['H2O/OH-'])], y=[-3.,-3.], 
                         marker=dict(symbol = "circle", size = 10, color=lc2, line_color="black", line_width=1)), row = r, col = c)
# Carbon
fig.add_trace(go.Scatter(mode = "lines", x=[convert_inset(D['CO2/CO2mol']),convert_inset(D['CO2/CO32-'])], y=[-4.,-4.], 
                         line_color = lc1, line_width = lw,line_dash = "solid"), row = r, col = c)
fig.add_trace(go.Scatter(mode = "markers", x=[convert_inset(D['CO2/CO2mol']),convert_inset(D['CO2/CO32-'])], y=[-4.,-4.], 
                         marker=dict(symbol = "circle", size = 10, color=lc1, line_color="black", line_width=1)), row = r, col = c)
# Sulfur
fig.add_trace(go.Scatter(mode = "lines", x=[convert_inset(D['H2S/S2-']),convert_inset(D['SO2/S6+'])], y=[-5.,-5.], 
                         line_color = lc3, line_width = lw,line_dash = "solid"), row = r, col = c)
fig.add_trace(go.Scatter(mode = "markers", x=[convert_inset(D['H2S/S2-']),convert_inset(D['SO2/S6+'])], y=[-5.,-5.], 
                         marker=dict(symbol = "circle", size = 10, color=lc3, line_color="black", line_width=1)), row = r, col = c)

# data examples
c=2
# Hydrogen
# Rescale hydrogen degassing example to "fit" data: initial H2O = 3 wt% and initial dD = -53 per mil
# H2O-OH- closed-system melt
fig.add_trace(go.Scatter(mode = "lines", x=hydrogen_OH['F']*3, y=hydrogen_OH['dm_closed']-53, 
                         line_color = lc4, line_width = lw,line_dash = "solid"), row = r, col = c)
# H2O-H2Omol closed-system melt
fig.add_trace(go.Scatter(mode = "lines", x=hydrogen_H2Omol['F']*3, y=hydrogen_H2Omol['dm_closed']-53, 
                         line_color = lc5, line_width = lw,line_dash = "solid"), row = r, col = c)
# H2O-OH- open-system melt
fig.add_trace(go.Scatter(mode = "lines", x=hydrogen_OH['F']*3, y=hydrogen_OH['dm_open']-53, 
                         line_color = lc4, line_width = lw,line_dash = "dash"), row = r, col = c)
# H2O-H2Omol open-system melt
fig.add_trace(go.Scatter(mode = "lines", x=hydrogen_H2Omol['F']*3, y=hydrogen_H2Omol['dm_open']-53, 
                         line_color = lc5, line_width = lw,line_dash = "dash"), row = r, col = c)
# H2O-OH-+H2Omol open-system melt model from Newman et al. (1988) Figure 6
fig.add_trace(go.Scatter(mode = "lines", x=newman88['H2O_open'], y=newman88['dD_open'], 
                         line_color = "black", line_width = lw,line_dash = "dash"), row = r, col = c)
# H2O-OH-+H2Omol closed-system melt model from Newman et al. (1988) Figure 6
fig.add_trace(go.Scatter(mode = "lines", x=newman88['H2O_closed'], y=newman88['dD_closed'], 
                         line_color = "black", line_width = lw,line_dash = "solid"), row = r, col = c)
# Matrix glass data from Newman et al. (1988)
fig.add_trace(go.Scatter(mode = "markers", x=newman88['H2O_wtpc'], y=newman88['dD_pm'], 
                         marker=dict(symbol = "circle", size = syms, color=lc2, line_color="black", line_width=1),
                         error_x=dict(type='data', array=newman88['H2O_wtpc_sd'],visible=True,width=0),
                         error_y=dict(type='constant', value=2.,visible=True,width=0)), 
                         row = r, col = c)
fig.update_yaxes(title = "\u03B4D (\u2030)",dtick= 50,title_font=dict(family="Times new roman"), range = [-150,0],side='right', row = r, col = c)
fig.update_xaxes(title = "H<sub>2</sub>O (wt%)",dtick= 0.5, range = [0,3], tickformat = ".1f", row = r, col = c)

r=2
c=1
# Carbon
# Rescale carbon degassing example to "fit" data: initial CO2 = 500 ppm and initial d13C = -8 per mil
# CO2-CO320 closed-system melt
fig.add_trace(go.Scatter(mode = "lines", x=carbonate['F']*500, y=carbonate['dm_closed']-8, 
                         line_color = lc5, line_width = lw,line_dash = "solid"), row = r, col = c)
# CO2-CO320 closed-system vapor
fig.add_trace(go.Scatter(mode = "lines", x=carbonate['F']*500, y=carbonate['dv_closed']-8, 
                         line_color = lc4, line_width = lw,line_dash = "solid"), row = r, col = c)
# CO2-CO320 open-system melt
fig.add_trace(go.Scatter(mode = "lines", x=carbonate['F']*500, y=carbonate['dm_open']-8, 
                         line_color = lc5, line_width = lw,line_dash = "dash"), row = r, col = c)
# CO2-CO320 open-system accumulated vapor
fig.add_trace(go.Scatter(mode = "lines", x=carbonate['F']*500, y=carbonate['dv_open']-8, 
                         line_color = lc4, line_width = lw,line_dash = "dash"), row = r, col = c)
# CO2-CO320 open-system instantaneous vapor
fig.add_trace(go.Scatter(mode = "lines", x=carbonate['F']*500, y=carbonate['dv_open_inst']-8, 
                         line_color = lc4, line_width = lw,line_dash = "dot"), row = r, col = c)
# Matrix glass data from Aubaud et al. (2004)
fig.add_trace(go.Scatter(mode = "markers", x=aubaud04['CO2_ppm'], y=aubaud04['d13C_m_pm'], 
                         marker=dict(symbol = "circle", size = syms, color=lc1, line_color="black", line_width=1)), 
                         row = r, col = c)
# Crushed vapor data from Aubuad et al. (2004)
fig.add_trace(go.Scatter(mode = "markers", x=aubaud04['CO2_ppm'], y=aubaud04['d13C_v_pm'], 
                         marker=dict(symbol = "diamond", size = syms, color=lc1, line_color="black", line_width=1)), 
                         row = r, col = c)
fig.update_yaxes(title = "\u03B4<sup>13</sup>C (\u2030)",title_font=dict(family="Times new roman"),dtick= 2, range = [-14,-4], row = r, col = c)
fig.update_xaxes(title = "CO<sub>2</sub> (ppm)",dtick= 100, range = [0,500], row = r, col = c)

c=2
# Sulfur
# Rescale sulfur degassing example to "fit" data: initial S = 1400 ppm and initial d13C = +1.5 per mil
# SO2-S6+ closed-system melt
fig.add_trace(go.Scatter(mode = "lines", x=sulfur_oxidised['F']*1400, y=sulfur_oxidised['dm_closed']+1.5, 
                         line_color = lc4, line_width = lw,line_dash = "solid"), row = r, col = c)
# H2S-S2- closed-system melt
fig.add_trace(go.Scatter(mode = "lines", x=sulfur_reduced['F']*1400, y=sulfur_reduced['dm_closed']+1.5, 
                         line_color = lc5, line_width = lw,line_dash = "solid"), row = r, col = c)
# SO2-S6+ open-system melt
fig.add_trace(go.Scatter(mode = "lines", x=sulfur_oxidised['F']*1400, y=sulfur_oxidised['dm_open']+1.5, 
                         line_color = lc4, line_width = lw,line_dash = "dash"), row = r, col = c)
# H2S-S2- open-system melt
fig.add_trace(go.Scatter(mode = "lines", x=sulfur_reduced['F']*1400, y=sulfur_reduced['dm_open']+1.5, 
                         line_color = lc5, line_width = lw,line_dash = "dash"), row = r, col = c)
# Melt inclusion and matrix glass data from Lerner et al. (2021)
fig.add_trace(go.Scatter(mode = "markers", x=lerner21['S_ppm'], y=lerner21['d34S_pm'], 
                         marker=dict(symbol = lerner21['symbol'], size = syms, color=lc3, line_color="black", line_width=1),
                         error_x=dict(type='data', array=lerner21['S_ppm_sd'],visible=True,width=0),
                         error_y=dict(type='data', array=lerner21['d34S_pm_sd'],visible=True,width=0)), 
                         row = r, col = c)
fig.update_yaxes(title = "\u03B4<sup>34</sup>S (\u2030)",title_font=dict(family="Times new roman"),dtick= 2, side='right', range = [-4,4], row = r, col = c)
fig.update_xaxes(title = "S (ppm)",dtick= 200, range = [0,1400], row = r, col = c)

# annotations
r=1
c=1
fig.add_annotation(x=0.95, y=1.6,text="(a)",showarrow=False, font=dict(color="black",size=15),textangle=0,  row = r, col = c)
fig.add_annotation(x=0.85, y=-3.4,text="OH<sup>-</sup>",showarrow=False,
                   ax = 0, ay = 0, font=dict(color="black"), textangle=0, row = r, col = c)
fig.add_annotation(x=0.35, y=-3.45,text="H<sub>2</sub>O<sub>mol</sub>",showarrow=False,
                   ax = 0, ay = 0, font=dict(color="black"), textangle=0, row = r, col = c)
fig.add_annotation(x=((0.35+0.85)/2), y=-3.4,text="[H<sub>2</sub>O]",showarrow=False,
                   ax = 0, ay = 0, font=dict(color="black"), textangle=0, row = r, col = c)
fig.add_annotation(x=0.37, y=-5,text="S<sup>6+</sup>",showarrow=False,
                   ax = 0, ay = 0, font=dict(color="black"), textangle=0, row = r, col = c)
fig.add_annotation(x=0.65, y=-5,text="S<sup>2-</sup> [SO<sub>2</sub>-H<sub>2</sub>S]",showarrow=False,
                   ax = 0, ay = 0, font=dict(color="black"), textangle=0, row = r, col = c)
fig.add_annotation(x=0.35, y=-4,text="CO<sub>2,mol</sub>",showarrow=False,
                   ax = 0, ay = 0, font=dict(color="black"), textangle=0, row = r, col = c)
fig.add_annotation(x=0.63, y=-4,text="CO<sub>3</sub><sup>2-</sup> [CO<sub>2</sub>]",showarrow=False,
                   ax = 0, ay = 0, font=dict(color="black"), textangle=0, row = r, col = c)
fig.add_annotation(x=0.45, y=-2.5,text="0",showarrow=True,
                   ax = 0, ay = -11, font=dict(color="black"), textangle=0, row = r, col = c)
fig.add_annotation(x=0.25, y=-2.5,text="-20",showarrow=True,
                   ax = 0, ay = -11, font=dict(color="black"), textangle=0, row = r, col = c)
fig.add_annotation(x=0.95, y=-2.5,text="+50",showarrow=True,
                   ax = 0, ay = -11, font=dict(color="black"), textangle=0, row = r, col = c)
fig.add_annotation(x=0.7, y=-2.1,text="\u0394 (\u2030)",showarrow=False,
                   ax = 0, ay = 0, font=dict(color="black",family='Times new roman'), textangle=0, row = r, col = c)
fig.add_annotation(x=0.12, y=1.7,text="\u2014 closed",showarrow=False,
                   ax = 0, ay = 0, font=dict(color='black'), textangle=0, row = r, col = c)
fig.add_annotation(x=0.12, y=1.3,text="\u2013 \u2013 open",showarrow=False,
                   ax = 0, ay = 0, font=dict(color='black'), textangle=0, row = r, col = c)
fig.add_annotation(x=0.18, y=0.9,text="\u22C5\u22C5\u22C5 open (inst.)",showarrow=False,
                   ax = 0, ay = 0, font=dict(color='black'), textangle=0, row = r, col = c)
fig.add_annotation(x=0.42, y=1.7,text="\u2014 melt",showarrow=False,
                   ax = 0, ay = 0, font=dict(color=lc1), textangle=0, row = r, col = c)
fig.add_annotation(x=0.435, y=1.3,text="\u2014 vapor",showarrow=False,
                   ax = 0, ay = 0, font=dict(color=lc2), textangle=0, row = r, col = c)
c=2
fig.add_annotation(x=2.85, y=-7,text="(b)",showarrow=False, font=dict(color="black",size=15),textangle=0,  row = r, col = c)
fig.add_annotation(x=1., y=-34,text="H<sub>2</sub>O<sub>mol</sub>-H<sub>2</sub>O",showarrow=False,
                   ax = 0, ay = 0, font=dict(color=lc5), textangle=0, row = r, col = c)
fig.add_annotation(x=0.7, y=-93,text="OH<sup>-</sup>-H<sub>2</sub>O",showarrow=False,
                   ax = 0, ay = 0, font=dict(color=lc5), textangle=0, row = r, col = c)
fig.add_annotation(x=0.57, y=-57,text="H<sub>2</sub>O<sub>mol</sub>+OH<sup>-</sup>-H<sub>2</sub>O",showarrow=False,
                   ax = 0, ay = 0, font=dict(color='black'), textangle=0, row = r, col = c)
fig.add_annotation(x=2.5, y=-140,text="\u25CF matrix glass",showarrow=False,
                   ax = 0, ay = 0, font=dict(color=lc2), textangle=0, row = r, col = c)
r=2
c=1
fig.add_annotation(x=25, y=-4.5,text="(c)",showarrow=False, font=dict(color="black",size=15),textangle=0,  row = r, col = c)
fig.add_annotation(x=430, y=-7.9,text="melt",showarrow=False,
                   ax = 0, ay = 0, font=dict(color=lc5), textangle=0, row = r, col = c)
fig.add_annotation(x=460, y=-4.9,text="vapor",showarrow=False,
                   ax = 0, ay = 0, font=dict(color=lc5), textangle=0, row = r, col = c)
fig.add_annotation(x=370, y=-13.,text="\u25CF matrix glass",showarrow=False,
                   ax = 0, ay = 0, font=dict(color=lc1), textangle=0, row = r, col = c)
fig.add_annotation(x=390, y=-13.5,text="\u25C6 crushed vesicles",showarrow=False,
                   ax = 0, ay = 0, font=dict(color=lc1), textangle=0, row = r, col = c)
c=2
fig.add_annotation(x=1330, y=3.6,text="(d)",showarrow=False, font=dict(color="black",size=15),textangle=0,  row = r, col = c)
fig.add_annotation(x=150, y=3.6,text="S<sup>6+</sup>-SO<sub>2</sub>",showarrow=False,
                   ax = 0, ay = 0, font=dict(color=lc5), textangle=0, row = r, col = c)
fig.add_annotation(x=200, y=-1.5,text="S<sup>2-</sup>-H<sub>2</sub>S",showarrow=False,
                   ax = 0, ay = 0, font=dict(color=lc5), textangle=0, row = r, col = c)
fig.add_annotation(x=1100, y=-3.0,text="\u25CF matrix glass",showarrow=False,
                   ax = 0, ay = 0, font=dict(color=lc3), textangle=0, row = r, col = c)
fig.add_annotation(x=1140, y=-3.5,text="\u25A0 melt inclusions",showarrow=False,
                   ax = 0, ay = 0, font=dict(color=lc3), textangle=0, row = r, col = c)

fig.update_layout(height=700, width=700, plot_bgcolor='rgb(255,255,255)' , showlegend = False)
fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True, ticks="inside", ticklen=5, title_standoff = 0, tickcolor="black",title_font=dict(size=15), tickfont=dict(size=12))
fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True, ticks="inside", ticklen=5, title_standoff = 0, tickcolor="black",title_font=dict(size=15), tickfont=dict(size=12))

fig.update_layout(font_family="Arial",font_color="black")

fig.show()

Caption: Volatile stable isotopes are fractionated during degassing. (a) The change in isotopic composition expected for melt (red) and vapor (blue) during degassing for closed (solid) and open (dash or dot) system degassing for a constant fractionation factor, where for the vapor the dash line is the integrated vapor and the dots are the instantaneous (inst.) vapor. The inset shows equilibrium fractionation factors between melt and vapor for H, C, and S. Examples of volatile stable isotope data for natural samples: (b) rhyolitic matrix glass from Mono Craters (California, USA); (c) basaltic matrix glass and associated vesicles from the Easter Microplate; and (d) basaltic matrix glass and melt inclusions from the 2018 eruption of Kīlauea (Hawaiʻi, USA).